In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00


In [ ]:
#@title Draft V.3 обработка по диалогам, ограниченный вывод первых 100 совпадений (разбитый).
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка модели и токенизатора
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_dialog_vector(dialog):
    # Токенизация всего диалога
    inputs = tokenizer(dialog, return_tensors='pt', truncation=True, padding=True, max_length=512)
    # Получение выходов модели
    with torch.no_grad():
        outputs = model(**inputs)
    # Усреднение скрытых состояний всех токенов в диалоге
    dialog_vector = outputs.last_hidden_state.mean(dim=1).numpy()
    return dialog_vector


In [ ]:
#@title Draft V.3 # Загрузка датасета
dataset_path = "//content/drive/MyDrive/Стажировка КИА/Диалоги/service.md"

with open(dataset_path, 'r') as file:
    # Извлечение диалогов и их уровней
    matches = re.findall(r'(#+)([^#]+)', file.read())
    levels = [len(match[0]) for match in matches]
    dialogs = [match[1].strip() for match in matches]

# Создание DataFrame из списка диалогов и их уровней
df_manager = pd.DataFrame({'dialog': dialogs, 'level': levels})

# Создание векторов для каждого диалога
df_manager['vector'] = df_manager['dialog'].apply(get_dialog_vector)

# Измерение косинусного сходства между векторами
similarities = cosine_similarity(np.concatenate(df_manager['vector'].values))



In [ ]:
#@title # Отбор близких диалогов
threshold = 0.95
selected_dialogs = []
for i, row in enumerate(similarities):
    for j, value in enumerate(row):
        if i != j and value > threshold:
            selected_dialogs.append((df_manager.iloc[i]['dialog'], df_manager.iloc[j]['dialog']))

# Создание DataFrame для вывода результатов
output_df = pd.DataFrame(selected_dialogs, columns=['Диалог', 'Близкий диалог'])



In [ ]:
#@title Draft V.3 вывод результатов

# Выводим количество близких диалогов
print(f"Количество близких диалогов: {len(output_df)}")

# Ограничение вывода первыми 100 совпадениями
#output_df = output_df.head(100)

# Запись результатов в текстовый файл
#output_df.to_csv('sep_dialog.txt', sep='\t', index=False, encoding='utf-8-sig')
output_df.to_csv('sep_dialog.txt', index=False, encoding='utf-8-sig')

# Проверка, что таблица сформирована
print(output_df)


Количество близких диалогов: 40
                                               Диалог  \
0   Первая замена масла\n Регламентное ТО автомоби...   
1   ТО автомобиля KIA RIO X и замена масла\n Регла...   
2   Расчет стоимости технического обслуживания ТО ...   
3   Расчет стоимости технического обслуживания ТО ...   
4   Наличие сервисной инспекции на автомобиль\nИнф...   
5   Неправильный гарантийный сервис для автомобиле...   
6   Запись на сервис через официальный дилерский ц...   
7   Процедура и стоимость ТО для Kia на сайте комп...   
8   Процедура и стоимость ТО для Kia на сайте комп...   
9   Процедура и стоимость ТО для Kia на сайте комп...   
10  Процедура и стоимость ТО для Kia на сайте комп...   
11  Процедура и стоимость ТО для Kia на сайте комп...   
12  Запрос на гарантийный осмотр\n Запись на гаран...   
13  Ограничение гарантии при превышении техобслужи...   
14  Стоимость ТО-5 для Kia Cerato на сайте Kia.ru ...   
15  Стоимость ТО-5 для Kia Cerato на сайте Kia.ru ...   

In [ ]:
#@title  # Отбор близких диалогов с объединением уникальныех элементов
threshold = 0.95
combined_dialogs = []
for i, row in enumerate(similarities):
    for j, value in enumerate(row):
        if i != j and value > threshold:
            combined = combine_dialogs(df_manager.iloc[i]['dialog'], df_manager.iloc[j]['dialog'])
            combined_dialogs.append((df_manager.iloc[i]['dialog'], df_manager.iloc[j]['dialog'], combined))

# Создание DataFrame для вывода результатов
output_df = pd.DataFrame(combined_dialogs, columns=['Диалог', 'Близкий диалог', 'Объединенный диалог'])


In [ ]:
#@title Draft V.3 # Функция объединения уникальных элементов двух диалогов
def combine_dialogs(dialog1, dialog2):
    # Разбиваем диалоги на предложения (или утверждения)
    sentences_1 = set(dialog1.split('\n'))
    sentences_2 = set(dialog2.split('\n'))

    # Объединяем уникальные элементы двух диалогов
    combined = sentences_1.union(sentences_2)

    return '\n'.join(combined)


In [ ]:
# инсталлируем библиотеку для работы с pdf

!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.1 MB/s eta 0:00:00


In [ ]:
# импортируем нужные модули

import pdfminer.high_level
import requests
import os
import re
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import numpy as np

##Функция №1
Эта функция, получает на вход URL-ссылку на PDF файл и переводит его в текст.

In [ ]:
'''
Эта функция принимает на вход URL-адрес
по которому находится pdf файл, переводит его в текстовый формат
и сохраняет на диск.

  Вход:
    url - адрес PDF файла
  Выход:
    topic - текст
'''

def pdf_to_txt(url):
  topic = ''
  pattern = re.compile(r'\w[а-я]')
  # получаем файл
  pdf_req = requests.get(url)
  # проверяем, что страница удачно парсится
  if pdf_req.status_code == 200:
    # запоминаем название файла без расширения
    f_name = os.path.basename(url).split('.')[0]
    # записываем файл на диск
    with open(f'{f_name}.pdf','wb') as f:
      f.write(pdf_req.content)
    # переводим файл в текст
    text = pdfminer.high_level.extract_text(f'{f_name}.pdf')
    # удаляем pdf файл
    if os.path.isfile(f'{f_name}.pdf'):
      os.remove(f'{f_name}.pdf')
    # исправляем некоторые ошибки парсинга
    text = text.replace(' \n', ' ')
    text = text.replace('-\n', '-')
    # разбиваем текст на строки для дальнейшей обработки
    txt_split = text.split('\n')
    # убираем пробелы в начале и конце каждой строки
    for st in range(len(txt_split)):
      res = txt_split[st].strip()
      # убираем повторяющиеся пробелы внутри строки
      res = re.sub('\s+',' ', res)
      # если строка не пустая и не номер страницы - добавляем её к тексту
      if res and not res.isdigit() and (len(res)>1):
        # это условие убирает ошибочные разрывы абзаца
        if re.match(r'[а-я]', res[0]):
          topic = topic[:-1]
          topic += ' '
        topic += res+'\n'
    # записываем текст в файл с названием оригинала, но txt
    with open(f'{f_name}.txt','w') as f:
      f.write(topic)

    return topic


In [ ]:
# пррверяем как работает функция на конкретном файле

url = 'https://cdn.kia.ru/master-data/brochures/Kia_Corporate_Sales.pdf'

print(pdf_to_txt(url))

Kia Бизнес
Сильные игроки для Вашей команды
Kia в мире
Kia Russia & CIS — корейская автомобильная корпорация, основанная в 1944 году. Ежегодно на 14 заводах Kia, расположенных в пяти странах мира, производится более 3 миллионов автомобилей.
Ежегодный доход
Дистрибьюторы и дилеры Kia
~$44 млрд в 180 странах
Общее число сотрудников
> 50 000 человек
Kia в России
Компания Kia Russia & CIS — официальный импортер и дистрибьютор автомобилей Kia в Российской Федерации. Начав работу в марте 2009 года, компания за короткий срок вывела бренд Kia в лидеры российского рынка.
Дилерская сеть Kia — самая большая в России
199 официальных дилеров в 100 городах России — от Калининграда до Петропавловска-Камчатского. Компания постоянно работает над развитием и расширением сети, чтобы быть ближе к клиентам и партнерам по всей стране.
Ежегодно с 2014 года Kia Russia & CIS занимает первое место среди иностранных автомобильных брендов в России по объему продаж.
Kia для бизнеса
Оптимальная стоимость, надежност

##Функция №2
Эта функция, получает на вход URL-ссылку на страницу сайта и выбирает из неё все ссылки на PDF файлы. Возвращает в виде словаря.

In [ ]:
def find_pdf(url):
  # Получаем запрос от страницы
  response = requests.get(url)
  # Получаем ПрекрасныйСуп
  bs = BeautifulSoup(response.text, "html.parser")

  # Определяем функцию отбора тегов. Берём теги у которых есть ссылка.
  def is_href(href):
    return href

  # Отбитаем теги у которых есть ссылка
  bs.find_all(href = is_href)

  # Заводим пустой словарь
  PDF_name = {}

  # Наполняем словарь описаниями файла и ссылками
  # То, что это словарь - важно. Так мы избегаем одинаковых файлов, но с разными ссылками. (такое есть)
  for tag in bs.find_all(href = is_href):
      # Извлекаем ссылку из тега
      tag_href = tag.attrs['href']
      # Делаем условие, если ссылка на файл, который оканчивается на .pdf
      if tag_href.split('.')[-1] == 'pdf':
        # Добавляем новый элемент в словарь
        PDF_name[tag.get_text().strip()] = tag_href
  # Возвращаем словарь
  return PDF_name

##Смотрим как работает функция на примере одного URL.

In [ ]:
# Смотрим как работает функция

# Берём произвольную страницу из БЗ
url = 'https://www.kia.ru/auction/'

# Делаем запрос и извлекаем название страницы
response = requests.get(url)
bs = BeautifulSoup(response.text, "html.parser")

# Извлекаем "словарь" (список PDFок с сайта)
PDFs = find_pdf(url)

# Условие: Что-то печаем, если PDF на странице сайта есть
if PDFs:
  # Если PDFки есть, то то печатаем заголовок
  print(f'Страница: {bs.title.string}')
  print(f'Адрес страницы: {url}')

  # Распечатываем PDF файлы, которые есть на этой странице
  for key,value in PDFs.items():
    print(f'PDF - "{key}"\n\tссылка: {value}')


Страница: Kia Аукцион: продай свой автомобиль Kia с максимальной выгодой
Адрес страницы: https://www.kia.ru/auction/
PDF - "Правила проведения Аукциона для Продавцов"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/rules_for_individuals.pdf
PDF - "Акт приема-передачи автомобиля"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/act_of_acceptance_transfer_of_the_car.pdf
PDF - "Шаблон договора купли-продажи автомобиля"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/sale_contract_template.pdf


##Функция №3
А теперь проходимся по всей базе и находим все PDFки.
Сначала подгружаем БД:


Поскольку я начал в парсинге заниматься PDF, то могу сказать, что в PDF информации даже чуть больше, чем на страницах.
Цикл проходтися по всем нашим ссылкам и выдирает все PDFки.

In [ ]:
# cкачиваем фалы данных в виде csv

def load_document_csv(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=tsv')
    response.raise_for_status()
    text = response.content

    return text

urls = load_document_csv('https://docs.google.com/spreadsheets/d/1h5BNBTXwzb8nMTIcre0vCDcujax1Fvfw3RdwT7FeAt8/edit?usp=sharing')
with open('urls_r.csv','wb') as f:
  f.write(urls)

In [ ]:
# Дальше загружаем из файла данных в массим Numpy

links = np.genfromtxt('urls_r.csv', delimiter='\t', dtype=str, encoding='utf-8' )

In [ ]:
# Идём по списку ссылок
for row in links:

  # Делаем запрос и извлекаем URL стреницы, который находится в столбце [1]
  response = requests.get(row[1])
  bs = BeautifulSoup(response.text, "html.parser")

  # Ищем все вхождения PDF (описание функции выше)
  PDFs = find_pdf(row[1])

  # Условие: Что-то печатаем, если PDF на странице сайта есть
  if PDFs:
    # Если PDFки есть, то печатаем заголовок
    print(f'\nСтраница: {bs.title.string}')
    print(f'Адрес страницы: {row[1]}')

    # Распечатываем PDF файлы, которые встречаются на страницах
    for key,value in PDFs.items():
      print(f'PDF - "{key}"\n\tссылка: {value}')



Страница: Kia Picanto 2022 – Киа Пиканто на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/picanto/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Picanto_Pre.pdf

Страница: Kia Rio 2022 – Седан Киа Рио на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/rio/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Rio_New.pdf

Страница: Kia Rio X 2022 – Кросс-хэтчбек Киа Рио Х на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/rio-x/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Rio_X_Preview_New.pdf

Страница: Kia Ceed 2022 – Хэтчбек Киа Сид на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/ceed/desc/
PDF - "Обзор функций (PDF)"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Connect_CD_PE.pdf
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Ceed_new_p

И теперь мы можем преобразовать все PDF файлы в текст при помощи Функции №1. Разделить его на чанки любым из доступных способов, а также придумать свой, оригинальный. ))